# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [1]:
#Import modules
import numpy as np #Data manipulation
import pandas as pd #Point data manipulation and organization
import xarray as xr #Raster data manipulation and organization

import pathlib  #For filepaths, io, etc.
import os       #For several system-based commands
import datetime #For manipulation of time data, including file creation/modification times
import json     #For dictionary io, etc.

import matplotlib.pyplot as plt #For plotting and data vizualization
import geopandas as gpd         #For organization and manipulation of vector data in space (study area and some data points)
import rioxarray as rxr         #For orgnaization and manipulation of raster data
import shapely                  #For converting coordinates to point geometry

#Scripts with functions made for this specific application
from lib import readData    #For reading data 
from lib import mapping     #For geospatial data manipulation
from lib import cleanData   #For cleaning well data
from lib import classify    #For classifying well data
from lib import exportData  #For exporting data

#Variables needed throughout, best to just assign now
todayDate, dateSuffix = readData.getCurrentDate() 
repoDir = pathlib.Path(os.getcwd())

# Read in data

- Set up filepaths
- Read in data from:
    - downholeData table (from database)
    - headerData table (from database)
    - xyzData file (from previously carried out work) (will eventually make this updateable)

Read in data

In [13]:
directoryDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\\'[:-1]

downholeDataPATH, headerDataPATH, xyzInPATH  = readData.filesSetup(db_dir=directoryDir)

#Functions to read data into dataframes. Also excludes extraneous columns, and drops header data with no location information
headerDataIN, downholeDataIN = readData.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH) 
xyzDataIN = readData.readXYZData(xyzfile=xyzInPATH)

Most Recent version of this file is : ISGS_DOWNHOLE_DATA_2023-01-06.txt
Most Recent version of this file is : ISGS_HEADER_2023-01-06.txt
Most Recent version of this file is : xyzData.csv
Using the following files:

\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_DOWNHOLE_DATA_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_HEADER_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\xyzData.csv
Downhole Data has 3054409 valid well records.
Header Data has 636855 unique wells with valid location information.


Define datatypes (doing this during the read in process has presented issues)

In [14]:
#Define datatypes, to read into defineDataTypes() function
##EVENTUALLY, MAKE THIS A FILE IN THE RES FOLDER TO READ IN
#downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
#headerDataDTYPES = {'ID':np.uint32,'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "ELEV_FT":np.float64}
#xyzDataDTYPES = {'ID':np.uint64, 'API_NUMBER':np.uint64, "LATITUDE":np.float64, "LONGITUDE":np.float64, "ELEV_FT":np.float64}

#Define datatypes of each column of the new dataframes
downholeDataIN = readData.defineDataTypes(downholeDataIN, dtypeFile='downholeDataTypes.txt')
headerDataIN = readData.defineDataTypes(headerDataIN, dtypeFile='headerDataTypes.txt')
xyzDataIN = readData.defineDataTypes(xyzDataIN, dtypeFile='xyzDataTypes.txt')

#Make a copy of the data so raw data is preserved while we work with the rest of the data
downholeData = downholeDataIN.copy()
headerData = headerDataIN.copy()
xyzData = xyzDataIN.copy()

Add in Control points

In [4]:
#NEED CODE HERE FOR ADDING IN CONTROL Wells MANUALLY
#Add control headerInfo
#Add control description info

# Extract Elevation Data

Extract elevation data from consistent elevation dataset for all wells (lidar or other statewide DEM)

In [5]:
#First, get wells with updated xyz info
    #Check first if xyzData needs to be updated with locations (?)
    #Check which wells in headerData don't have associated lidar data

#statewideLidar =  ow
#mapping.rastertoPoints_extract()

Merge elevation data with headerData table

In [15]:
uniqueWells = headerData['API_NUMBER'].unique()
#xyzData['UniqueWells'] = uniqueWells

headerData = mapping.addElevtoHeader(xyzData, headerData)
##NEED TO UPDATE THIS TO WORK WITH DATA WITH NO XYZ ELEVATION DATA FROM LIDAR
#Change xyz column name to indicate lidar
#Use order of preference: lidar, headerData table?/30/10m DEM?

# Data Cleaning

## First, let's clean up records in the data without the necessary information

Clip data from outside Study Area

Read in Study Area

In [16]:
studyAreaPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESL_StudyArea_5mi.shp"
studyAreaIN, saExtent = mapping.readStudyArea(studyAreaPath)

In [25]:
#headerDataClip = gpd.clip(headerData, studyArea_4269) #Data from table is in EPSG:4269, easier to just project study area to ensure data fit
print(headerData.crs)
studyArea_4269

EPSG:4269


,Id,geometry
0,0,"POLYGON ((-90.33797 38.57354, -90.33809 38.573..."


In [18]:
headerData = mapping.coords2Geometry(df=headerData, xCol='LONGITUDE', yCol='LATITUDE', crs='EPSG:4269')
#headerData['geometry']=headerData['GEOMETRY'].copy() #old code
studyArea_4269 = studyAreaIN.to_crs('EPSG:4269').copy()
headerDataClip = gpd.clip(headerData, studyArea_4269) #Data from table is in EPSG:4269, easier to just project study area to ensure data fit
headerDataClip

c:\ProgramData\Anaconda3\envs\raster38\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT,geometry


Now, remove data from downholeData table that does not have location information (Since we would not know where to put it anyway)

This should also essentially "clip" the downholeData to the study area, since only study area wells remain in headerData

In [ ]:
downholeData = cleanData.removeNonlocatedData(downholeData, headerData)

3054409 records removed without location information.
0 wells remain from 0 located wells in study area.


In [ ]:
headerData

,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT,geometry


Remove headerData rows without surface elevation information (this currently clips data from outside Illinois)

In [ ]:
headerData = cleanData.removenotopo(headerData, printouts=True)

Number of rows before dropping those without surface elevation information: 0
Number of rows after dropping those without surface elevation information: 0
Well records deleted: 0


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [ ]:
#Drop records with no depth information
donwholeData = cleanData.dropnodepth(downholeData, printouts=True)
#Drop records with bad depth information (i.e., top depth > bottom depth) (Also calculates thickness of each record)
donwholeData = cleanData.dropbaddepth(downholeData, printouts=True)

Number of rows before dropping those without record depth information: 0
Number of rows after dropping those without record depth information: 0
Number of well records without formation information deleted: 0
Number of rows before dropping those with obviously bad depth information: 0
Number of rows after dropping those with obviously bad depth information: 0
Well records deleted: 0


Drop records with no FORMATION information

In [ ]:
downholeData = cleanData.dropnoformation(downholeData, printouts=True)

Number of rows before dropping those without FORMATION information: 0
Number of rows after dropping those without FORMATION information: 0
Well records deleted: 0


Now we are going to export this data, to have record of cleaned data

In [ ]:
downholeData.reset_index(inplace=True,drop=True)
headerData.reset_index(inplace=True,drop=True)

#downholeData.to_csv(str(repoDir)+'/out/downholeData_cleaned'+dateSuffix+'.csv',index_label='ID')
#headerData.to_csv(str(repoDir)+'/out/headerData_cleaned'+dateSuffix+'.csv',index_label='ID')

# Classification

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: Wildcard match (startTerm) - no context
- 3: Bedrock classification for obvious bedrock
- 4: Wildcard match (startTerm) - with context

In [ ]:
#Read in dictionary files for downhole data
specTermsPATH, startTermsPATH = readData.searchTermFilePaths()

Most Recent version of this file is : SearchTerms-Specific_2022-11-16_essCols.csv
Most Recent version of this file is : SearchTerms-Start.csv


In [ ]:
specTerms, startTerms = readData.readSearchTerms(specfile=specTermsPATH, startfile=startTermsPATH)

Join the dataframes--for the specific search terms, this is the same as classifying them

In [ ]:
downholeData_spec = classify.specificDefine(downholeData, specTerms, printouts=True)
downholeData = downholeData_spec.copy()

Records Classified with full search term: 0
Records Classified with full search term: nan% of data


c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\lib\classify.py:21: RuntimeWarning: invalid value encountered in longlong_scalars
  print("Records Classified with full search term: "+str(round((df_Interps['CLASS_FLAG'].sum()/df_Interps.shape[0])*100,2))+"% of data")


Create a dataframe with only the records already classified (using the specific search terms in this case, classifiedDF), and one that still needs to be searched (searchDF)

In [ ]:
classifedDF, searchDF = classify.splitDefined(downholeData)
searchDF.shape[0]

0

Now, do the classification routine on the searchDF database

In [ ]:
searchDF = classify.startDefine(df=searchDF, starterms=startTerms, printouts=True)

Start Term process should be done by 14:00
Records classified with start search term: 0
Records classified with start search term: nan% of remaining data


c:\Users\balikian\LocalData\CodesScripts\Github\wells4hydrogeology\lib\classify.py:44: RuntimeWarning: invalid value encountered in longlong_scalars
  print("Records classified with start search term: "+str(round((df['CLASS_FLAG'].count()/df.shape[0])*100,2))+"% of remaining data")


Merge specDF and searchDF back together all back in single dataframe

In [ ]:
downholeData_Terms = classify.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Terms.copy()

Export terms that still need to be defined to csv (along with their counts)

In [ ]:
classify.export_toBeDefined(downholeData, str(repoDir)+'/out/')

Classify all  data under depth threshold (default is 550') as bedrock (should not be an issue, but just in case)

In [ ]:
classifedDF, searchDF = classify.splitDefined(downholeData)
searchDF = classify.depthDefine(searchDF, thresh=550, printouts=True)
downholeData_Class = classify.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Class.copy()

Series([], Name: CLASS_FLAG, dtype: int64)


KeyError: 3

Add '0' flag for data still not classified

In [ ]:
downholeData = classify.fillUnclassified(downholeData)

In [ ]:
downholeData['CLASS_FLAG'].value_counts()

1.0    2138359
0.0     716206
4.0     107010
3.0      79219
Name: CLASS_FLAG, dtype: int64

## Add "Flag" for target interpratations

In [ ]:
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
targetInterpDF = readData.readLithologies()

In [ ]:
downholeData = classify.mergeLithologies(downholedata=downholeData, targinterps=targetInterpDF)

Flags used for target classification purposes:
- -2: No classification 
- -1: Classified, not used/not definitive
- 0: Classified, not target material
- 1: Classified as target material

In [ ]:
downholeData['TARGET'].value_counts()

-1    1456486
-2     796686
0      549662
1      237960
Name: TARGET, dtype: int64

Find all unique wells in downhole dataset

In [ ]:
#Get Unique well APIs
wellsDF = classify.getUniqueWells(downholeData)

Number of unique wells in downholeData: 458260


Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [ ]:
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True)
downholeData_sorted

,index,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG,TARGET,Unnamed: 2,MUD
0,274385,10915812,FORMATION_TOPS,Herrin Coal #6,5.0,400.0,405.0,BEDROCK,1.0,True,-1,NaN,DIRT AND GRAVEL
1,274386,10915912,FORMATION_TOPS,Herrin Coal #6,6.0,403.0,409.0,BEDROCK,1.0,True,-1,NaN,DIRT AND GRAVEL
2,274387,10916012,FORMATION_TOPS,Herrin Coal #6,6.0,406.0,412.0,BEDROCK,1.0,True,-1,NaN,DIRT AND GRAVEL
3,274388,10916112,FORMATION_TOPS,Danville Coal #7,3.0,390.0,393.0,BEDROCK,1.0,True,-1,NaN,DIRT AND GRAVEL
4,274389,10916112,FORMATION_TOPS,Herrin Coal #6,6.0,404.0,410.0,BEDROCK,1.0,True,-1,NaN,DIRT AND GRAVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040789,274370,4289724608,HWYBRIDGE_LOG,"medium, very damp, sandy clay loam with 2"" thi...",2.5,39.5,42.0,NaN,0.0,False,-2,NaN,NaN
3040790,274381,4289724608,HWYBRIDGE_LOG,"very loose, wet, fine grain, sand",4.0,42.0,46.0,NaN,0.0,False,-2,NaN,NaN
3040791,274374,4289724608,HWYBRIDGE_LOG,"soft, wet, sandy clay loam with 1/2"" thick san...",2.5,46.0,48.5,NaN,0.0,False,-2,NaN,NaN
3040792,274376,4289724608,HWYBRIDGE_LOG,"stiff, damp, cohesive mixture of sand, clay, g...",2.0,48.5,50.5,NaN,0.0,False,-2,NaN,NaN


# Get Bedrock Depth and Layer Thickness

Read in/Define Model Grid

In [ ]:
modelGridPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\grid_625_raster.tif"
modelGrid = mapping.readModelGrid(saExtent=saExtent, gridpath=modelGridPath, nodataval=0, readGrid=True)

Read in surface elevation grid

In [ ]:
surfaceElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ILStateLidar_ClipExtentESL.tif"
surfaceElevGridIN = mapping.readSurfaceGrid(surfaceelevpath=surfaceElevPath, useWCS=False)

UnboundLocalError: local variable 'noDataSurf' referenced before assignment

Read in bedrock elevation grid

In [ ]:
bedrockElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESLBedrock.tif"
bedrockElevGridIN=mapping.readBedrockGrid(bedrockelevpath=bedrockElevPath)

Plot just to see them

In [ ]:
fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockElevGridIN.plot(ax=ax[0])
surfaceElevGridIN.plot(ax=ax[1])

Reproject and align raster grids for surface elevation and bedrock topo (reproject well data too if needed)

In [ ]:
bedrockGrid, surfaceGrid = mapping.alignRasters(bedrockgrid=bedrockElevGridIN, surfacegrid=surfaceElevGridIN, modelgrid=modelGrid)

fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockGrid.plot(ax=ax[0])
surfaceGrid.plot(ax=ax[1])

Use the surface elevation raster and bedrock elevation raster to get depth to bedrock

In [ ]:
driftThickGrid, layerThickGrid = mapping.getDriftThick(surface=surfaceGrid, bedrock=bedrockGrid, noLayers=9, plotData=True)

Now, sample each well point (headerData) to get layer thickness, surface elevation, and bedrock 

In [ ]:
headerData = mapping.rastertoPoints_sample(raster=bedrockGrid, ptDF=headerData, newColName='BEDROCK_ELEV_FT')
headerData['BEDROCK_ELEV_M'] = headerData['BEDROCK_ELEV_FT']* 0.3048

headerData = mapping.rastertoPoints_sample(raster=surfaceGrid, ptDF=headerData, newColName='SURFACE_ELEV_FT')
headerData['SURFACE_ELEV_M'] = headerData['SURFACE_ELEV_FT']* 0.3048

headerData = mapping.rastertoPoints_sample(raster=layerThickGrid, ptDF=headerData, newColName='LAYER_THICK_FT')
headerData['LAYER_THICK_M'] = headerData['LAYER_THICK_FT']* 0.3048

Calculate  all layer depths/elevations at all wells

In [ ]:
noLayers = 9
for layer in range(0, noLayers): #For each layer
    #Make column names
    depthColName  = 'Depth_FT_LAYER'+str(layer)
    depthMcolName = 'Depth_M_LAYER'+str(layer) 

    elevColName = 'ELEV_FT_LAYER'+str(layer)
    elevMColName = 'ELEV_M_LAYER'+str(layer)
    
    #Calculate depth to each layer at each well, in feet and meters
    headerData[depthColName]  = headerData['Layer_Thick_FT'] * layer
    headerData[depthMcolName] = headerData[depthColName] * 0.3048
    
    headerData[elevColName]  = headerData['SURFACE_ELEV_FT'] - headerData['Layer_Thick_FT'] * layer
    headerData[elevMColName]  = headerData['SURFACE_ELEV_M'] - headerData['Layer_Thick_M'] * layer


# Work here next!

Define function to calculate target thickness in each layer

In [ ]:
##THIS CELL MAY NEED TO BE UPDATED!!!!!


#Define the function to export the result of thickness of target sediments in each layer
def Layers_surfacedown(df, layer = 1):
    
    #Generate Column names based on (looped) integers
    topCol = "ESL_ModelTopoLyrs_"+str(layer)
    if layer != 9: #For all layers except the bottom layer....
        botCol = "ESL_ModelTopoLyrs_"+str(layer+1) #use the layer below it to 
    else: #Otherwise, ...
        botCol = "BedrockCorr" #Use the (corrected) bedrock depth

    #Divide records into 4 separate categories for ease of calculation, to be joined back together later  
        #Category 1: Well interval starts above layer top, ends within model layer
        #Category 2: Well interval is entirely contained withing model layer
        #Category 3: Well interval starts within model layer, continues through bottom of model layer
        #Category 4: well interval begins and ends on either side of model layer (model layer is contained within well layer)

    #records1 = intervals that go through the top of the layer and bottom is within layer
    records1 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records1['TARG_THICK'] = pd.DataFrame(np.round((records1.loc[:,topCol]-records1.loc[: , 'BOT_ELEV_ft']) * records1['Target'],3)).copy()
    
    #records2 = entire interval is within layer
    records2 = df.loc[(df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] >= df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records2['TARG_THICK'] = pd.DataFrame(np.round((records2.loc[: , 'TOP_ELEV_ft'] - records2.loc[: , 'BOT_ELEV_ft']) * records2['Target'],3)).copy()

    #records3 = intervals with top within layer and bottom of interval going through bottom of layer
    records3 = df.loc[(df['TOP_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records3['TARG_THICK'] = pd.DataFrame(np.round((records3.loc[: , 'TOP_ELEV_ft'] - (records3.loc[:,botCol]))*records3['Target'],3)).copy()

    #records4 = interval goes through entire layer
    records4 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records4['TARG_THICK'] = pd.DataFrame(np.round((records4.loc[: , topCol]-records4.loc[: , botCol]) * records4['Target'],3)).copy()

    
    #Put the four calculated record categories back together into single dataframe
    res = records1.append(records2).append(records3).append(records4)
    
    res_df = res.groupby(['API_NUMBER','LATITUDE','LONGITUDE'],as_index=False).sum()#calculate thickness for each well interval in the layer indicated (e.g., if there are two well intervals from same well in one model layer)

    res_df['TARG_THICK_PER'] = pd.DataFrame(np.round(res_df['TARG_THICK']/res_df['LyrThick'],3)) #Calculate thickness as percent of total layer thickness
    res_df["LAYER"] = layer #Just to have as part of the output file, include the present layer in the file itself as a separate column
    res_df = res_df[['API_NUMBER', 'LATITUDE', 'LONGITUDE', 'TOP', 'BOTTOM','SURF_ELEV_ft', 'TOP_ELEV_ft', 'BOT_ELEV_ft',topCol,botCol,'LyrThick','TARG_THICK', 'TARG_THICK_PER', 'LAYER']].copy() #Format dataframe for output
    
    return res, res_df

Now run that function over all the layers, looping through each one

In [ ]:
#THIS CELL WILL NEED TO BE UPDATED

outDIR = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\MetroEast_HydroGeo\\CodeOutput\\"+codeTarget+"\\"

for i in np.arange(1,10):
    res, res_df = Layers_surfacedown(df, layer = i)#Run the function defined above for each layer
    outputname = codeTargShort+'Lyr'+str(i)+'.csv' #Create a filename based on the layer and target
    res_df.to_csv(outDIR+outputname)  #Export the file to csv
    #Could also potentially save these to variables for use in following cells

NameError: name 'codeTarget' is not defined

# Interpolate thickness values in each layer

Loop through each layer and interpolate (use same parameters (?))

Ensure rasters align (are co-registered) with grid

# Export

In [ ]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")